In [1]:
from google.cloud import bigquery
client=bigquery.Client()

In [3]:
import pandas as pd

In [9]:
import numpy as np
import json

In [5]:
sql="""
WITH 
cvar_embs as (
SELECT postId, signal, lang, embs FROM `maximal-furnace-783.sc_hamsa_logging.hamsa-insert-events-logging` WHERE DATE(time) = "2023-04-12" 
AND SOURCE IN ('sc-warmup-cold-job')
),
scposts as (
select * from `maximal-furnace-783.sc_analytics.sharechat_tv_source_metadata` WHERE DATE(time) = "2023-04-12" and language='Hindi'
)
SELECT a.postId,signal,lang,embs,IF(b.postId IS NULL,0,1) as ScPost FROM cvar_embs as a
LEFT JOIN scposts as b
on a.postId=b.postId
"""
cvar_emb=client.query(sql).to_dataframe()

In [6]:
cvar_emb.to_csv("cvar_embs_sctv__normal.csv")

In [43]:
"""Deprecated"""
# cvar_emb["emb_arr"]=cvar_emb["embs"].apply(lambda x:json.loads(x))

'Deprecated'

In [28]:
'''Convert column to a numpy array to calculate the norm'''
cvar_emb["np_emb"]=cvar_emb["embs"].apply(lambda x:np.array(eval(x)),0)
cvar_emb["norm"]=cvar_emb["np_emb"].apply(np.linalg.norm)

In [34]:
cvar_emb.head(10)

,postId,signal,lang,embs,ScPost,emb_arr,np_emb,norm
0,5892087055,fav,Hindi,"[0.21280501782894135,0.08881976455450058,0.173...",0,"[0.21280501782894135, 0.08881976455450058, 0.1...","[0.21280501782894135, 0.08881976455450058, 0.1...",1.052720
1,5303087055,fav,Hindi,"[0.1397312432527542,0.17300429940223694,0.3861...",0,"[0.1397312432527542, 0.17300429940223694, 0.38...","[0.1397312432527542, 0.17300429940223694, 0.38...",1.112366
2,1733087055,fav,Hindi,"[-0.05496601387858391,0.1584838181734085,0.092...",0,"[-0.05496601387858391, 0.1584838181734085, 0.0...","[-0.05496601387858391, 0.1584838181734085, 0.0...",1.349001
3,9143087055,fav,Hindi,"[0.24752165377140045,0.20885202288627625,0.323...",0,"[0.24752165377140045, 0.20885202288627625, 0.3...","[0.24752165377140045, 0.20885202288627625, 0.3...",1.155904
4,7620677055,fav,Hindi,"[0.3238089382648468,0.15732915699481964,0.1376...",0,"[0.3238089382648468, 0.15732915699481964, 0.13...","[0.3238089382648468, 0.15732915699481964, 0.13...",0.775850
5,7529977055,fav,Hindi,"[0.145242,0.132724,0.078676,0.112505,-0.00912,...",0,"[0.145242, 0.132724, 0.078676, 0.112505, -0.00...","[0.145242, 0.132724, 0.078676, 0.112505, -0.00...",0.666058
6,5328177055,fav,Hindi,"[0.00013699941337108612,0.09182636439800262,-0...",0,"[0.00013699941337108612, 0.09182636439800262, ...","[0.00013699941337108612, 0.09182636439800262, ...",1.502738
7,7419977055,fav,Hindi,"[0.140473,0.090039,0.137656,0.144603,0.094285,...",0,"[0.140473, 0.090039, 0.137656, 0.144603, 0.094...","[0.140473, 0.090039, 0.137656, 0.144603, 0.094...",0.736988
8,1904087055,fav,Hindi,"[0.1446203589439392,0.0637279525399208,0.47732...",0,"[0.1446203589439392, 0.0637279525399208, 0.477...","[0.1446203589439392, 0.0637279525399208, 0.477...",1.338307
9,5770087055,fav,Hindi,"[0.145242,0.132724,0.078676,0.112505,-0.00912,...",0,"[0.145242, 0.132724, 0.078676, 0.112505, -0.00...","[0.145242, 0.132724, 0.078676, 0.112505, -0.00...",0.666058


In [42]:
cvar_emb.groupby(["ScPost"]).agg({'postId':'count'})

,postId
ScPost,
0,4462693
1,28046


In [36]:
cvar_emb.groupby(["ScPost","signal"]).agg({'norm':['mean','std']})

norm          
                   mean       std
ScPost signal                    
0      fav     0.994998  0.257480
       like    1.299075  0.248335
       lpo     0.491127  0.176653
       share   0.893513  0.261013
       vclick  1.220930  0.360844
       vplay   0.940889  0.256108
       vskip   1.121830  0.336183
1      fav     1.071736  0.295950
       like    1.331715  0.218898
       share   0.743061  0.172258
       vclick  0.903784  0.538341
       vplay   0.706316  0.314843
       vskip   0.933839  0.553170

In [37]:
cvar_emb.groupby(["ScPost"]).agg({'norm':['mean','std']})

norm          
            mean       std
ScPost                    
0       1.022049  0.348667
1       0.958134  0.425080